**Task 07: Querying RDF(s)**

In [ ]:
!pip install rdflib 
github_storage = "https://raw.githubusercontent.com/FacultadInformatica-LinkedData/Curso2020-2021/master/Assignment4"

     |████████████████████████████████| 235kB 5.1MB/s 
     |████████████████████████████████| 51kB 6.5MB/s 


Read the RDF file

In [ ]:
from rdflib import Graph, Namespace, Literal
from rdflib.namespace import RDF, RDFS
g = Graph()
g.namespace_manager.bind('ns', Namespace("http://somewhere#"), override=False)
g.namespace_manager.bind('vcard', Namespace("http://www.w3.org/2001/vcard-rdf/3.0#"), override=False)
g.parse(github_storage+"/resources/example6.rdf", format="xml")

<Graph identifier=Na004e274461e43b0a308a84a07c69fcb (<class 'rdflib.graph.Graph'>)>

**TASK 7.1: List all subclasses of "Person" with RDFLib and SPARQL**

**sparql**

In [ ]:
from rdflib.plugins.sparql import prepareQuery

ns = Namespace("http://somewhere#")

q1 = prepareQuery('''
  SELECT 
    ?sc
  WHERE { 
    ?sc rdfs:subClassOf/rdfs:subClassOf* ns:Person. 
  }
  ''',
  initNs = { "ns": ns}
)


for r in g.query(q1):
  print(r)

(rdflib.term.URIRef('http://somewhere#Researcher'),)
(rdflib.term.URIRef('http://somewhere#PhDstudent'),)


**rdflib**

In [ ]:
for s,p,o in g.triples((None, RDFS.subClassOf, ns.Person)):
    print(s)
for ss,pp,oo in g.triples((None, RDFS.subClassOf, s)): 
  print(ss)

http://somewhere#Researcher
http://somewhere#PhDstudent


**TASK 7.2: List all individuals of "Person" with RDFLib and SPARQL (remember the subClasses)**


**sparql**

In [ ]:
q2 = prepareQuery('''
  SELECT 
    ?s 
  WHERE { 
    {?s a ns:Person.}
    UNION {
      ?sc rdfs:subClassOf/rdfs:subClassOf* ns:Person.
      ?s rdf:type ?sc
    }  
  }
  ''',
  initNs = { "ns": ns}
)


for r in g.query(q2):
  print(r)

(rdflib.term.URIRef('http://somewhere#SaraJones'),)
(rdflib.term.URIRef('http://somewhere#JohnSmith'),)
(rdflib.term.URIRef('http://somewhere#JaneSmith'),)


**rdflib**

In [ ]:
for s,p,o in g.triples((None, RDF.type, ns.Person)):
  print(s)
for sc,p,o in g.triples((None, RDFS.subClassOf, ns.Person)):
  for s,p,o in g.triples((None, RDF.type, sc)):
    print(s)

http://somewhere#SaraJones
http://somewhere#JohnSmith
http://somewhere#JaneSmith


**TASK 7.3: List all individuals of "Person" and all their properties including their class with RDFLib and SPARQL**


**sparql**

In [ ]:
q3 = prepareQuery('''
  SELECT 
    ?s ?p
  WHERE { 
    {
    ?s a ns:Person. 
    ?s ?p ?o
    }
    UNION {
      ?sc rdfs:subClassOf/rdfs:subClassOf* ns:Person.
      ?s rdf:type ?sc.
      ?s ?p ?o
    }  
  }
  ''',
  initNs = { "ns": ns}
)

for r in g.query(q3):
  print(r.s,r.p)

http://somewhere#SaraJones http://www.w3.org/1999/02/22-rdf-syntax-ns#type
http://somewhere#SaraJones http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Given
http://somewhere#SaraJones http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Family
http://somewhere#SaraJones http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/FN
http://somewhere#JohnSmith http://www.w3.org/1999/02/22-rdf-syntax-ns#type
http://somewhere#JohnSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Family
http://somewhere#JohnSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/FN
http://somewhere#JohnSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Given
http://somewhere#JaneSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/FN
http://somewhere#JaneSmith http://www.w3.org/1999/02/22-rdf-syntax-ns#type
http://somewhere#JaneSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Given
http://somewhere#JaneSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Famil

**rdflib**

In [16]:

print("imprimo los que son de tipo persona")
for s,p,o in g.triples((None, RDF.type, ns.Person)):
  for ss,po,oo in g.triples((s,None,None)):
    print(ss,pp,oo)

print("imprimo los que son de subtipo o que son de otro tipo que es de tipo persona")
for sc,p,o in g.triples((None, RDFS.subClassOf, ns.Person)):
  for s,p,o in g.triples((None, RDF.type, sc)):
    for ss,pp,oo in g.triples((s,None,None)):
      print(ss,pp,oo)

imprimo los que son de tipo persona
http://somewhere#SaraJones http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Family http://somewhere#Person
http://somewhere#SaraJones http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Family Sara
http://somewhere#SaraJones http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Family Jones
http://somewhere#SaraJones http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Family Sara Jones
http://somewhere#JohnSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Family http://somewhere#Person
http://somewhere#JohnSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Family Smith
http://somewhere#JohnSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Family John Smith
http://somewhere#JohnSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Family John
imprimo los que son de subtipo o que son de otro tipo que es de tipo persona
http://somewhere#JaneSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/FN jane Sm